In [1]:
using LinearAlgebra

"""
    eliminacion_gaussiana_pivoteo_parcial(A_aumentada)

Resuelve un sistema de ecuaciones lineales n x n utilizando el algoritmo de
eliminación gaussiana con pivoteo parcial y seguimiento de filas simulado.

# Argumentos
- `A_aumentada::Matrix{Float64}`: La matriz aumentada del sistema [A|b] de
  tamaño n x (n+1).

# Retorna
- `Vector{Float64}`: El vector de solución `x` si existe una solución única.
- `nothing`: Si el sistema no tiene una solución única, e imprime un mensaje.
"""
function eliminacion_gaussiana_pivoteo_parcial(A_aumentada::Matrix{Float64})
    # Hacemos una copia para no modificar la matriz original
    A = copy(A_aumentada)
    n = size(A, 1) # Número de ecuaciones

    # Paso 1: Inicializar el vector de índices de fila
    NROW = collect(1:n)

    println("--- Iniciando Eliminación Gaussiana con Pivoteo Parcial ---")
    println("Matriz Original:")
    display(A)
    println()

    # --- PROCESO DE ELIMINACIÓN ---

    # Paso 2: Bucle principal para cada columna/pivote
    for i in 1:(n-1)
        # Paso 3: Encontrar la fila 'p' con el máximo pivote en la columna 'i'
        # Buscamos desde la fila i hasta la n
        vals_columna = abs.(A[NROW[i:n], i])
        _, p_local = findmax(vals_columna)
        p = i + p_local - 1 # Ajustamos el índice al vector NROW completo

        # Paso 4: Verificar si el pivote máximo es cero
        if A[NROW[p], i] == 0
            println("Error: No existe una solución única.")
            return nothing # SALIDA
        end

        # Paso 5: Intercambio de fila simulado (si es necesario)
        if NROW[i] != NROW[p]
            println("Pivote en columna $i: |$(A[NROW[p], i])|. Intercambiando (simulado) fila $i con $p.")
            NROW[i], NROW[p] = NROW[p], NROW[i]
        end
        
        # Paso 6: Bucle para las filas debajo del pivote
        for j in (i+1):n
            # Paso 7: Calcular el multiplicador
            m = A[NROW[j], i] / A[NROW[i], i]

            # Paso 8: Ejecutar la eliminación en la fila j
            # Se actualiza la fila real en la matriz A, usando los índices de NROW
            A[NROW[j], :] .-= m .* A[NROW[i], :]
        end
    end
    
    println("\n--- Matriz Modificada (forma triangular superior, filas desordenadas) ---")
    display(A)
    println("\nOrden final de las filas (NROW): ", NROW)
    println()


    # Paso 9: Verificar si existe una solución única después de la eliminación
    if A[NROW[n], n] == 0
        println("Error: No existe una solución única.")
        return nothing # SALIDA
    end

    # --- INICIA SUSTITUCIÓN HACIA ATRÁS ---

    x = zeros(Float64, n)

    # Paso 10: Determinar x_n
    x[n] = A[NROW[n], n+1] / A[NROW[n], n]

    # Paso 11: Bucle hacia atrás para las demás incógnitas
    for i in (n-1):-1:1
        # Sumatoria de a_ij * x_j usando los índices de NROW
        suma = dot(A[NROW[i], (i+1):n], x[(i+1):n])
        
        x[i] = (A[NROW[i], n+1] - suma) / A[NROW[i], i]
    end

    # Paso 12: Devolver la solución
    println("--- Procedimiento completado con éxito ---")
    return x # SALIDA
end

eliminacion_gaussiana_pivoteo_parcial

In [2]:
# Sistema de ecuaciones:
# 0x₁ + 2x₂ + 1x₃ = 5
# 4x₁ + 1x₂ - 1x₃ = 3
# 1x₁ + 1x₂ + 3x₃ = 6

# Matriz aumentada [A|b]
matriz_problematica = [0.0 2.0 1.0 5.0;
                       4.0 1.0 -1.0 3.0;
                       1.0 1.0 3.0 6.0]

# Llamamos a la función
solucion = eliminacion_gaussiana_pivoteo_parcial(matriz_problematica)

# Imprimimos el resultado si existe
if solucion !== nothing
    println("\nLa solución del sistema es:")
    println("x₁ = ", round(solucion[1], digits=4))
    println("x₂ = ", round(solucion[2], digits=4))
    println("x₃ = ", round(solucion[3], digits=4))

    # Verificación
    println("\nVerificación A*x (debe ser aprox. igual a b):")
    b_calculado = matriz_problematica[:, 1:3] * solucion
    display(b_calculado)
end

--- Iniciando Eliminación Gaussiana con Pivoteo Parcial ---
Matriz Original:


3×4 Matrix{Float64}:
 0.0  2.0   1.0  5.0
 4.0  1.0  -1.0  3.0
 1.0  1.0   3.0  6.0


Pivote en columna 1: |4.0|. Intercambiando (simulado) fila 1 con 2.

--- Matriz Modificada (forma triangular superior, filas desordenadas) ---


3×4 Matrix{Float64}:
 0.0  2.0   1.0    5.0
 4.0  1.0  -1.0    3.0
 0.0  0.0   2.875  3.375


Orden final de las filas (NROW): [2, 1, 3]

--- Procedimiento completado con éxito ---

La solución del sistema es:
x₁ = 0.5652
x₂ = 1.913
x₃ = 1.1739

Verificación A*x (debe ser aprox. igual a b):


3-element Vector{Float64}:
 5.0
 3.000000000000001
 6.000000000000001